In [1]:
#Guide here: https://stmorse.github.io/journal/espn-fantasy-v3.html
#https://github.com/cwendt94/ff-espn-api
%matplotlib inline
#The necessary packages
import requests
import pandas as pd
import numpy as np
import seaborn as sns
from ff_espn_api import League
#Info:
league_id=4570371
season=2019
#team='10'

In [2]:
league=League(league_id,2019)
myTeam=league.teams[9]

In [11]:
#This will be updated as the info is added
league.teams

[Team(Blue Bearies),
 Team(Team 2),
 Team(Team 3),
 Team(Team 4),
 Team(Team 5),
 Team(Team 6),
 Team(Team 7),
 Team(Team 8),
 Team(Team 9),
 Team(Rakdos Hackrobats),
 Team(Lil Chubb'rs),
 Team(Team 12)]

In [5]:
#Another version:
#Get the JSON table in easily readable format
#Basic team info
url='https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/segments/0/leagues/{:}'.format(league_id)
r=requests.get(url)
dat=r.json()

In [ ]:
#Get the weekly matchup scores
slotcodes={0:'QB',2:'RB',4:'WR',
          6:'TE',16:'Def',17:'K',
          20:'Bench',21:'IR',23:'Flex'
          }
url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + \
      str(season) + '/segments/0/leagues/' + str(league_id) + \
      '?view=mMatchup&view=mMatchupScore'


In [ ]:
data=[]
print('Week ',end='')
for week in range(1,2):
    print(week,end=' ')
    r=requests.get(url,params={'scoringPeriodId':week})
    d=r.json()
    for tm in d['teams']:
        tmid=tm['id']
        for p in tm['roster']['entries']:
            name=p['playerPoolEntry']['player']['fullName']
            slot=p['lineupSlotId']
            pos=slotcodes[slot]
            
            inj='NA'
            try:
                inj=p['playerPoolEntry']['player']['injuryStatus']
            except:
                pass
        proj,act=None,None
        for stat in p['playerPoolEntry']['player']['stats']:
            if stat['scoringPeriodId']!=week:
                continue
            if stat['statSourceId'] == 0:
                act = stat['appliedTotal']
            elif stat['statSourceId']==1:
                proj=stat['appliedTotal']
        data.append([
                week, tmid, name, slot, pos, inj, proj, act
        ])

In [ ]:
data = pd.DataFrame(data, 
                    columns=['Week', 'Team', 'Player', 'Slot', 
                             'Pos', 'Status', 'Proj', 'Actual'])